In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
import cv2
import numpy as np
from PIL import Image, ImageTk


class ImageProcessor:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Processor")
        self.root.geometry("400x600")

        self.image_label = tk.Label(root)
        self.image_label.pack(pady=10)

        self.load_button = tk.Button(root, text="Load Image", command=self.load_image)
        self.load_button.pack(fill=tk.X, padx=20, pady=5)

        self.crop_frame = tk.Frame(root)
        self.crop_frame.pack(fill=tk.X, padx=20, pady=5)
        self.crop_label = tk.Label(self.crop_frame, text="Crop (x, y, width, height):")
        self.crop_label.grid(row=0, column=0, padx=5, pady=5)
        self.crop_entry = tk.Entry(self.crop_frame)
        self.crop_entry.grid(row=0, column=1, padx=5, pady=5)

        self.process_button = tk.Button(root, text="Process Image", command=self.process_image)
        self.process_button.pack(fill=tk.X, padx=20, pady=5)

        self.filter_frame = tk.LabelFrame(root, text="Filter Options")
        self.filter_frame.pack(fill=tk.X, padx=20, pady=5)
        self.filter_var = tk.StringVar()
        self.filter_var.set("Select Filter")
        self.filter_menu = tk.OptionMenu(self.filter_frame, self.filter_var, "Select Filter", "Blur", "Sharpen", "Emboss")
        self.filter_menu.pack(fill=tk.X, padx=5, pady=5)

        self.rotate_frame = tk.Frame(root)
        self.rotate_frame.pack(fill=tk.X, padx=20, pady=5)
        self.rotate_label = tk.Label(self.rotate_frame, text="Rotate Angle:")
        self.rotate_label.grid(row=0, column=0, padx=5, pady=5)
        self.rotate_entry = tk.Entry(self.rotate_frame)
        self.rotate_entry.grid(row=0, column=1, padx=5, pady=5)

        self.watermark_frame = tk.Frame(root)
        self.watermark_frame.pack(fill=tk.X, padx=20, pady=5)
        self.watermark_label = tk.Label(self.watermark_frame, text="Watermark Text:")
        self.watermark_label.grid(row=0, column=0, padx=5, pady=5)
        self.watermark_entry = tk.Entry(self.watermark_frame)
        self.watermark_entry.grid(row=0, column=1, padx=5, pady=5)

        self.button_frame = tk.Frame(root)
        self.button_frame.pack(fill=tk.X, padx=20, pady=5)
        self.save_button = tk.Button(self.button_frame, text="Save Image", command=self.save_image)
        self.save_button.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=5, pady=5)
        self.save_cropped_button = tk.Button(self.button_frame, text="Save Cropped", command=self.save_cropped_image)
        self.save_cropped_button.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=5, pady=5)

        self.status_bar = tk.Label(root, text="Select an image to get started.", bd=1, relief=tk.SUNKEN, anchor=tk.W)
        self.status_bar.pack(side=tk.BOTTOM, fill=tk.X, padx=20, pady=5)

        self.original_image = None
        self.processed_image = None

    def load_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.original_image = cv2.imread(file_path)
            self.display_image(self.original_image)
            self.status_bar.config(text="Image loaded successfully.")

    def display_image(self, image):
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        self.image = ImageTk.PhotoImage(image=image)
        self.image_label.configure(image=self.image)
        self.image_label.image = self.image

    def process_image(self):
        if self.original_image is None:
            messagebox.showerror("Error", "No image loaded.")
            return

        crop_info = self.crop_entry.get().split(',')
        if len(crop_info) != 4:
            messagebox.showerror("Error", "Invalid crop format. Please use 'x, y, width, height'.")
            return
        try:
            x, y, width, height = map(int, crop_info)
            cropped_image = self.original_image[y:y + height, x:x + width]
        except ValueError:
            messagebox.showerror("Error", "Invalid crop coordinates. Please enter integers.")
            return

        action = self.filter_var.get()
        if action == "Blur":
            processed_image = cv2.GaussianBlur(cropped_image, (5, 5), 0)
        elif action == "Sharpen":
            processed_image = cv2.filter2D(cropped_image, -1, np.array([[-1, -1, -1],
                                                                       [-1, 9, -1],
                                                                       [-1, -1, -1]]))
        elif action == "Emboss":
            kernel = np.array([[0, -1, -1],
                               [1, 0, -1],
                               [1, 1, 0]])
            processed_image = cv2.filter2D(cropped_image, -1, kernel)
        else:
            processed_image = cropped_image

        rotate_angle = self.rotate_entry.get()
        if rotate_angle:
            try:
                angle = float(rotate_angle)
                rows, cols, _ = processed_image.shape
                rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
                processed_image = cv2.warpAffine(processed_image, rotation_matrix, (cols, rows))
            except ValueError:
                messagebox.showerror("Error", "Invalid rotation angle. Please enter a valid floating-point number.")

        watermark_text = self.watermark_entry.get()
        if watermark_text:
            font = cv2.FONT_HERSHEY_SIMPLEX
            position = (50, 50)  # Adjust the position as needed
            font_scale = 1
            color = (0, 0, 255)  # Red color for watermark
            thickness = 2
            cv2.putText(processed_image, watermark_text, position, font, font_scale, color, thickness)

        self.processed_image = processed_image
        self.display_image(processed_image)
        self.status_bar.config(text="Image processed successfully.")

    def save_image(self):
        if self.processed_image is not None:
            save_path = filedialog.asksaveasfilename(defaultextension=".jpg")
            if save_path:
                cv2.imwrite(save_path, self.processed_image)
                self.status_bar.config(text="Image saved successfully.")

    def save_cropped_image(self):
        if self.processed_image is not None:
            save_path = filedialog.asksaveasfilename(defaultextension=".jpg")
            if save_path:
                cv2.imwrite(save_path, self.processed_image)
                self.status_bar.config(text="Cropped image saved successfully.")


root = tk.Tk()
image_processor = ImageProcessor(root)
root.mainloop()
